## Principal Component Analysis (PCA) of Basin Characteristics

In estimating runoff in ungauged basins, candidate "proxy" basins are sought on the basis of similarity of the mechanisms governing runoff.  Intuitively, adjacent basins of similar size, shape, aspect, topology, land cover, geology, etc. are expected to generate similar runoff in volume and duration, if not timing.  Based on some measure of similarity, in this case daily average flow, are there basin characteristics that are consistently better predictors of similarity?  Are there combinations of characteristics, or principal components, that form a more meaningful basis to describe similarity?  In this example, patterns are sought in a large sample of watersheds in Canada featuring a common set of basin characteristics.  

### Data

The basins included in the dataset are a subset of the HYSETS database with HYDAT as the source, as these represent basins operated by the Water Survey of Canada.  A later step of the analysis evaluates runoff and basin characteristics for pairs of basins with a minimum length of concurrent data. The HYSETS database contains 2375 basins, and of all ($5.6E6$) basin pair combinations, $2.3E6$ basin pairs meet a minimum of 40 years of concurrent daily flow data.  There are 1030 unique basins in the set of all basin pairs meeting the minimum concurrence.  Of the remaining basins, 759 have a complete set of 15 basin characteristics.  (include discussion about choice of minimum concurrence, provide dataset with varying concurrence threshold criteria.)

### Methodology

The dataset is constructed of $N$ rows corresponding to unique basins and $m$ columns corresponding basin characteristics, represented by the matrix $$\textbf{Y} = \begin{bmatrix}y_{11} & y_{12} & \dots & y_{1m} \\ y_{21} & y_{22} & \dots & y_{2m} \\ \dots & \dots & \dots & \dots \\ y_{N1} & y_{N2} & \dots & y_{Nm} \end{bmatrix}$$

PCA is used to extract uncorrelated modes of variability in the data, on the basis that some more fundamental set of variables $e_1, e_2, \dots$ exists and describes the data (potentially) more concisely.  Following the method of Hotelling (1933), the data is standardized such that each variable (column) has zero mean and unit variance.  Given $m$ variables and $N$ samples, the objective is to find a matrix $$\textbf{e} = \begin{bmatrix}e_{11} & e_{12} & \dots & e_{1m} \\ e_{21} & e_{22} & \dots & e_{2m} \\ \dots & \dots & \dots & \dots \\ e_{m1} & e_{m2} & \dots & e_{mm} \end{bmatrix}$$ such that $$\textbf{Y} - \mathbf{\bar{Y}}  = \sum_{j=1}^{m} \textbf{a}_j \textbf{e}_j, \quad j = 1, 2, \dots, m$$  

The eigenvector columns $\textbf{e}_j$ describe the rotated coordinate axes of the PC modes, while the $\textbf{a}_j$ principal component "scores", or the "signal" of each variable, represented in the rotated space.  The modes describe the dynamics of the system in combinations of the original variables, and in descending order of fractional variance, that is the proportional amount of the total variance explained by the mode.  The eigenvectors by definition align with the axis yielding the greatest variance, however this does not necessarily mean the eigenvector will be aligned meaningfully with the data.  As a result, the PC scores ...

Plot the squared sum of square PC score (differences between pairs) vs. similarity?  Do we expect this to be any different than the unrotated space?  

In [ ]:
import time

import pandas as pd
import numpy as np

from minisom import MiniSom
import somoclu

from itertools import combinations

import matplotlib.pyplot as plt
from multiprocessing import Pool

from shapely.geometry import Point
import geopandas as gpd
import geoviews as gv
from geoviews import opts, tile_sources as gvts

import scipy.stats as st
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA, FastICA

from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral3
from bokeh.layouts import gridplot, column, row
from bokeh.io import output_file, save, show

import holoviews as hv
from holoviews import opts
from holoviews import dim
hv.extension('bokeh')

output_notebook()

## Do PCA on Characteristics of Individual Basins

In [ ]:
all_pair_results_df = pd.read_pickle('results/filtered_pairs_all_concurrent_lengths.csv')

unique_stations = all_pair_results_df[['b1', 'b2']].to_numpy().flatten()

unique_stations = list(set(unique_stations))

In [ ]:
basin_df = pd.read_csv('data/HYSETS_watershed_properties.txt', sep=';', dtype={'Official_ID': str})
basin_df = basin_df[basin_df['Source'] == 'HYDAT']
basin_df = basin_df[basin_df['Official_ID'].isin(unique_stations)]

In [ ]:
basin_pca_cols = [ 'Centroid_Lat_deg_N', 'Centroid_Lon_deg_E', 'Drainage_Area_km2', 'Elevation_m','Slope_deg', 
                  'Gravelius', 'Perimeter', 'Aspect_deg', 'Land_Use_Forest_frac', 'Land_Use_Grass_frac', 
                  'Land_Use_Wetland_frac', 'Land_Use_Water_frac', 'Land_Use_Urban_frac', 'Land_Use_Shrubs_frac', 
                  'Land_Use_Crops_frac', 'Land_Use_Snow_Ice_frac', 'Permeability_logk_m2', 'Porosity_frac']

# exclude any basins that are missing basin characteristics that will be used in 
basins_missing_data = basin_df[basin_df[basin_pca_cols].isnull().any(1)]['Official_ID'].to_numpy()

basin_df = basin_df[~basin_df[basin_pca_cols].isnull().any(1)]

basin_df.reset_index(inplace=True)
basin_df['index'] = basin_df.index

In [ ]:
# remove the two basins that are missing data.  Not sure how these got in here in the first place, but...
print(len(all_pair_results_df))
print(basins_missing_data)
all_pair_results_df = all_pair_results_df[(~all_pair_results_df['b1'].isin(basins_missing_data)) & (~all_pair_results_df['b2'].isin(basins_missing_data))]
print(len(all_pair_results_df))

In [ ]:
# create a dict to map the Official ID to the index
idx_id_dict = basin_df[['Official_ID', 'index']].set_index('Official_ID').to_dict()['index']

In [ ]:
# normed_df = (rdf - rdf.min()) / (rdf.max() - rdf.min())
# normed_df.dropna(inplace=True)
# print(f'Analyzing {len(raw_df)} basin pairs.')
print(f'Analyzing {len(basin_df)} basins.')
# print(basin_df.head())
# filter for the basin characteristic columns
basin_PCA_df = basin_df[basin_pca_cols].copy()

# standardize the input data
basin_PCA_df = (basin_PCA_df - basin_PCA_df.mean()) / basin_PCA_df.std()

# check mean and stdev are 0 and 1, respectively
# print(basin_PCA_df.mean())
# print(basin_PCA_df.std())

In [ ]:
ax = basin_PCA_df.plot(figsize=(12, 5))

ax.set_xlabel('basin # [-]')
ax.set_ylabel('value [-]')
plt.show()

In [ ]:
def create_grid_plot(data):
    j = 0
    figs = []
    for char in data.columns:
        p = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa",
                  title=char)
        x = data[char].to_numpy()
        
        hist, edges = np.histogram(x, density=True, bins='auto')
        
        
        p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
        figs.append(p)
    return figs
            
hist_grid = create_grid_plot(df)

In [ ]:
show(gridplot(hist_grid, ncols=4, plot_width=200, plot_height=130))

In [ ]:
# do PCA 
x = basin_PCA_df.index.values
y = basin_PCA_df.to_numpy()

n_modes = np.shape(y)[1] #dimension of input = 4, so want 4 modes

pca = PCA(n_components = n_modes)
PCs = pca.fit_transform(y)
eigvecs = pca.components_
fracVar = pca.explained_variance_ratio_

In [ ]:
print('Input shape = ', np.shape(y))
print('PC shape = ', np.shape(PCs))
print('Eigenvector shape', np.shape(eigvecs))
print('PCA Output Eigenvectors (first):')
print(eigvecs[0])

In [ ]:
# Sort eigenvectors and add labels for the eigenvector plot
def sort_eigenvectors(eigvecs, pca_variables):
    rank_dict = {}
    ev_num = 0
    for ev in eigvecs:
        sorted_evs_idx = ev.argsort()
        sorted_evs = ev[sorted_evs_idx][::-1]
        reordered_pca_vars = np.array(pca_variables)[sorted_evs_idx][::-1]
        rank_dict[ev_num] = {k: v for k, v in zip(reordered_pca_vars, sorted_evs)}
        ev_num += 1
    return rank_dict



In [ ]:
rank_dict =  sort_eigenvectors(eigvecs, basin_pca_cols)

In [ ]:
#plot fraction of variance explained by each mode

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.scatter(range(len(fracVar)),fracVar)
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by All Modes')

plt.subplot(1,2,2)
n_modes_show = 4
plt.scatter(range(n_modes_show),fracVar[:n_modes_show])
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by First ' + str(n_modes_show) + ' Modes')

plt.tight_layout()

plt.show()

In [ ]:
cumulative_var = 0
for n in range(len(fracVar)):
    cumulative_var += fracVar[n]
    print(f'Mode {n}: {fracVar[n]:.2f} (cumulative {cumulative_var:.2f})')

In [ ]:
#plot the first n modes and PCs 
n = PCs.shape[1]
plt.rcParams['savefig.facecolor'] = "0.8"
plt.figure(figsize=(9,4*n))
for kk in range(n):
    
    plt.subplot(n,2,kk*2+1)
#     plt.plot(eigvecs[kk,:])
    plt.plot(rank_dict[kk].keys(), rank_dict[kk].values())
    plt.title(f'Eigenvector of Mode #{kk+1}')
    plt.xticks(rotation=90)
    
    plt.subplot(n,2,(kk+1)*2)
    plt.plot(PCs[:,kk])
    plt.title(f'PCs of Mode #{kk+1}')
    plt.xlabel('Signal')
    
    plt.tight_layout()
#     plt.savefig('results/img/ordered_eigvecs_characteristics.png', transparent=False)

## Show pairs of variables in 2-d eigenspace

In [ ]:
pc_df = pd.DataFrame(PCs, columns=[f'PC {e}' for e in range(PCs.shape[1])])
pca_component_grid = create_grid_plot(pc_df)


In [ ]:
pc_df.head()

In [ ]:
show(gridplot(pca_component_grid, ncols=4, plot_width=200, plot_height=130))

In [ ]:
def make_PC_gridplot(PCs, n_modes):
    mode_pairs = list(combinations(list(range(n_modes)), 2))
    figs = []
    for pair in mode_pairs:
        

        p = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa",
                  title=f'{pair}')

        x1 = PCs[:, pair[0]]
        x2 = PCs[:, pair[1]]

        p.circle(x1, x2, alpha=0.5)
        figs.append(p)
    return figs
    
    
pc_space_figs = make_PC_gridplot(PCs, 9)    

In [ ]:
show(gridplot(pc_space_figs, ncols=6, plot_width=150, plot_height=130))

## Calculate euclidean distance of PC scores for basin pairs



In [ ]:
hysets_folder = '/media/danbot/T7 Touch/hysets_series/'

In [ ]:
def create_band(rdf, param, n_bins):
       
    if param == 'similarity':
        domain = 'normed_distance'
    else:
        domain = 'similarity'
        
#     hist, bin_edges = np.histogram(rdf[domain], density=True, bin_edges=b_edges)
#     n_samples_per_bin = 200
#     n_bins_100_sample = int(len(rdf) / n_samples_per_bin)
#     b_edges1 = st.mstats.mquantiles(rdf[param], np.linspace(0., 1.0, n_bins_100_sample))
    b_edges = st.mstats.mquantiles(rdf[param], np.linspace(0., 1.0, n_bins))
#     print(b_edges)
    b_edges = np.where(np.isnan(b_edges), 1, b_edges)
#     print(b_edges)
    
    tdf = pd.DataFrame()
    tdf['bin_edges'] = b_edges

    median_vals, lbound1, hbound1 = [], [], []
    bin_vals, lbound2, hbound2 = [], [], []

    for i in range(1,len(tdf)):
        min_edge = tdf.loc[i-1, 'bin_edges']
        max_edge = tdf.loc[i, 'bin_edges']
        mid_bin = (min_edge + max_edge) / 2
        bin_vals.append(mid_bin)
        grouped_vals = rdf[(rdf[param] >= min_edge) & (rdf[param] < max_edge)][domain]
#         print(f'{len(grouped_vals)} values between {min_edge:.5f} and {max_edge:.5f}')
        
        if len(grouped_vals) < 1:
            median_vals.append(np.nan)
            lbound1.append(np.nan)
            hbound1.append(np.nan)
            lbound2.append(np.nan)
            hbound2.append(np.nan)

        else:
            median_vals.append(np.percentile(grouped_vals, 50))
            lbound1.append(np.percentile(grouped_vals, 5))
            hbound1.append(np.percentile(grouped_vals, 95))
            lbound2.append(np.percentile(grouped_vals, 33.33333))
            hbound2.append(np.percentile(grouped_vals, 66.666667))
            
            
    band = pd.DataFrame()
    band['edge'] = bin_vals

    band['median'] = median_vals
    band['lbnd1'] = lbound1
    band['hbnd1'] = hbound1

    band['lbnd2'] = lbound2
    band['hbnd2'] = hbound2
#     print('')
  


    return band
    

In [ ]:
 #output to static HTML file
def create_fig(band, rdf, param):
    
    title = f"Expected similarity metric by equiprobable distance metric binning ({len(rdf)} basin pairs)"
    if param == 'similarity':
        title = f"Expected distance metric by equiprobable similarity metric binning ({len(rdf)} basin pairs)"

    p = figure(plot_width=600, plot_height=550, tools="wheel_zoom,box_zoom,pan,reset,lasso_select",
               title=title,
              match_aspect=True, background_fill_color='#440154')

    p.grid.visible = False
    
#     bins = hexbin(rdf['normed_dist'], rdf['similarity'], 0.05)
#     print(bins)
    
#     cmap = linear_cmap('counts', 'Viridis256', 0, max(bins.counts))

#     h = p.hex_tile(q="q", r="r", size=0.01, line_color=None, source=bins,
#                fill_color=cmap)
    
    p.circle(rdf['normed_distance'], rdf['similarity'], color="white", size=1,
            alpha=0.6)
    
    rdf.dropna(inplace=True)
    
    if param == 'similarity':

        domain = 'normed_distance'
        p.harea(band['lbnd1'], band['hbnd1'], 
                 band['edge'], alpha=0.3, color='white')
        p.harea(band['lbnd2'], band['hbnd2'], 
                 band['edge'], alpha=0.5, color='white')
        
        sim_all = np.percentile(rdf[domain], 50)
        sim_lb1 = np.percentile(rdf[domain], 5)
        sim_lb2 = np.percentile(rdf[domain], 33.33333)
        sim_hb1 = np.percentile(rdf[domain], 95)
        sim_hb1 = np.percentile(rdf[domain], 66.66667)
        
        y_vals = np.linspace(0, 1, 20)
        x_vals = [sim_all for _ in y_vals]
#         print(sim_all)
        p.line(x_vals, y_vals, color='#F012BE', line_width=3, 
              legend_label='EV (all data)', line_dash='dashed')
        
        p.line(band['median'], band['edge'], color='#F012BE', 
               line_width=2, legend_label='EV (bin)')
        
    else:

        domain = 'similarity'
        p.varea(band['edge'], band['lbnd1'], band['hbnd1'], 
                 alpha=0.3, color='white')
        p.varea(band['edge'], band['lbnd2'], band['hbnd2'], 
                 alpha=0.5, color='white')
        
        similarity_all = np.percentile(rdf['normed_distance'], 50)
        

        sim_all = np.percentile(rdf[domain], 50)
        sim_lb1 = np.percentile(rdf[domain], 5)
        sim_lb2 = np.percentile(rdf[domain], 33.33333)
        sim_hb1 = np.percentile(rdf[domain], 95)
        sim_hb1 = np.percentile(rdf[domain], 66.66667)
    
        x_vals = np.linspace(0, 1, 20)
        y_vals = [sim_all for _ in x_vals]
        

        
        p.line(x_vals, y_vals, color='#F012BE', line_width=3,
              legend_label='EV (all data)', line_dash='dashed')
        
        p.line(band['edge'], band['median'], color='#F012BE',
              line_width=2, legend_label='EV (bin)')

#     hover = HoverTool(
#         tooltips=[('distance', '@normed_distance'),
#                  ()],
#         mode='mouse',
#         point_policy='follow_mouse',
#         renderers=[h],
#     )
#     p.add_tools(hover)
        
#     min_d = rdf['normed_distance'].min()
#     max_d = rdf['normed_distance'].max()

    p.xaxis.axis_label = f'Euclidean Distance of Parameter Differences (1 = most dissimilar)'
    p.yaxis.axis_label = f'Similarity Metric (COD/R^2)'
    
    return p

In [ ]:
def create_layout(normed_df, n_bins, modes):
    similarity_df = create_band(normed_df, 'similarity', n_bins).dropna()
    dist_df = create_band(normed_df, 'normed_distance', n_bins).dropna()
    p1 = create_fig(similarity_df, normed_df, 'similarity')
    p2 = create_fig(dist_df, normed_df, 'normed_distance')
#     print(dist_df.head())
    
#     return (dist_df['edge'].to_numpy(), dist_df['median'].to_numpy())
    return row(p1, p2)

In [ ]:
n_bins  = 50

mode = 0 # PC mode

# band = create_band(pair_df, 'normed_distance', n_bins).dropna()

# layout = create_layout(pair_df, 50, 1)

In [ ]:
# show(layout)

## Plot the PC scores in Space

**NOTE:** this is for the station PCs only, not basin pair comparison

In [ ]:
spdf = pc_df.copy()
# spdf.drop(columns=['Unnamed: 0'], inplace=True)

spdf['geometry'] = basin_df.apply(lambda row: Point(row['Centroid_Lon_deg_E'],	row['Centroid_Lat_deg_N']), axis=1)
# spdf['lon'] = basin_df['Centroid_Lon_deg_E']
# spdf['lat'] = basin_df['Centroid_Lat_deg_N']
# gdf = 
# spdf.head()

gdf = gpd.GeoDataFrame(spdf, crs='EPSG:4326')
gdf = gdf.to_crs(3857)
# gdf.head()

In [ ]:
def calculate_PC_dist(row, modes):
    cols = [f'PC {m}' for m in modes]
    return np.sqrt((row[cols]**2).sum())
    

In [ ]:
modes_of_interest = [0]

gdf['stn_ID'] = basin_df['Official_ID']
gdf['PC_distance'] = gdf.apply(lambda row: calculate_PC_dist(row, modes_of_interest), axis=1)


# retrieve the centroid lat-lon and convert to shapely Point
points = hv.Points(gdf, vdims=['PC_distance', 'stn_ID'])

# x_range = (spdf.min()['lon'], spdf.max()['lon'])
# y_range = (spdf.min()['lat'], spdf.max()['lat'])

# tl = hv.element.tiles.StamenTerrain().redim.range()

gvts.EsriTerrain.opts(width=900, height=550) * points.opts(color='PC_distance', tools=['hover'], 
                                                           size=4, colorbar=True, 
                                                           title=f'PC score (mode set {modes_of_interest})')

## Now do PCA for station pairs

In [ ]:
cdf = pd.read_csv('results/results_min_365d_concurrent.csv')

In [ ]:
print(type(cdf['pair_midpoint'][0]))

In [ ]:
pair_pca_df = cdf.copy()
pair_pca_chars = [c for c in cdf.columns if c not in ['similarity', 'b1', 'b2', 'concurrent_days', 'pair_midpoint']]
print(char_cols)

In [ ]:
def do_PCA(df, char_cols):
        
    # standardize to zero mean and unit stdev (but exclude similarity column)
    df.loc[:,char_cols] = (df.loc[:, char_cols] - df.loc[:, char_cols].mean()) / df.loc[:, char_cols].std()

    # do PCA 
    x = df.index.values
    y = df[char_cols].to_numpy()

    n_modes = np.shape(y)[1] #full dimension of input

    pca = PCA(n_components = n_modes)
    PCs = pca.fit_transform(y)
    eigvecs = pca.components_
    fracVar = pca.explained_variance_ratio_
    return PCs, eigvecs, fracVar

In [ ]:
pair_PCs, pair_eigvecs, pair_fracVar = do_PCA(cdf, pair_pca_chars)

In [ ]:
pair_cumulative_var = 0
for n in range(len(pair_fracVar)):
    pair_cumulative_var += pair_fracVar[n]
    print(f'Mode {n}: {pair_fracVar[n]:.2f} (cumulative {pair_cumulative_var:.2f})')

In [ ]:
#plot fraction of variance explained by each mode
n_modes_show = pair_PCs.shape[1]
# pair_eigvec_dict = sort_eigenvectors(pair_eigvecs)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.scatter(range(len(pair_fracVar)),pair_fracVar)
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by All Modes')

plt.subplot(1,2,2)
n_modes_show = 4
plt.scatter(range(n_modes_show),pair_fracVar[:n_modes_show])
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by First ' + str(n_modes_show) + ' Modes')

plt.tight_layout()

plt.show()

In [ ]:
#plot the first n modes and PCs for ~90% of variance
n = pair_PCs.shape[1]
pair_rank_dict = sort_eigenvectors(pair_eigvecs, pair_pca_chars)
# plt.rcParams['savefig.facecolor'] = "0.8"
plt.figure(figsize=(9,4*n))
for kk in range(n):
    
    plt.subplot(n,2,kk*2+1)
#     plt.plot(eigvecs[kk,:])
    plt.plot(pair_rank_dict[kk].keys(), pair_rank_dict[kk].values())
    plt.title(f'Eigenvector of Mode #{kk}')
    plt.xticks(rotation=90)
    
    plt.subplot(n,2,(kk+1)*2)
    plt.plot(pair_PCs[:,kk])
    plt.title(f'PCs of Mode #{kk}')
    plt.xlabel('Signal')
    
    plt.tight_layout()
    plt.savefig('results/img/ordered_eigvecs_PCscores.png', transparent=False)

In [ ]:
pair_pc_df = pd.DataFrame(pair_PCs, columns=[f'PC {i}' for i in range(pair_PCs.shape[1])])
pair_pc_df['similarity'] = cdf['similarity'].copy()
pair_pc_df.head()


In [ ]:
def create_band(data, n_bins):
        # Use alpha, beta = 0.33, 0.33 according to
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.mquantiles.html
    b_edges = st.mstats.mquantiles(data['PC_distance_norm'], np.linspace(0., 1.0, n_bins),
                                  alphap=1./3, betap=1./3)
#     print(b_edges)
    b_edges = np.where(np.isnan(b_edges), 1, b_edges)
#     print(b_edges)
    
    tdf = pd.DataFrame()
    tdf['bin_edges'] = b_edges

    median_vals, lbound1, hbound1 = [], [], []
    bin_vals, lbound2, hbound2 = [], [], []

    for i in range(1,len(tdf)):
        min_edge = tdf.loc[i-1, 'bin_edges']
        max_edge = tdf.loc[i, 'bin_edges']
        mid_bin = (min_edge + max_edge) / 2
        bin_vals.append(mid_bin)
        grouped_vals = data[(data['PC_distance_norm'] >= min_edge) & (data['PC_distance_norm'] < max_edge)]['similarity']
#         print(f'{len(grouped_vals)} values between {min_edge:.5f} and {max_edge:.5f}')
        
        if len(grouped_vals) < 1:
            median_vals.append(np.nan)
            lbound1.append(np.nan)
            hbound1.append(np.nan)
            lbound2.append(np.nan)
            hbound2.append(np.nan)

        else:
            median_vals.append(np.percentile(grouped_vals, 50))
            lbound1.append(np.percentile(grouped_vals, 5))
            hbound1.append(np.percentile(grouped_vals, 95))
            lbound2.append(np.percentile(grouped_vals, 33.33333))
            hbound2.append(np.percentile(grouped_vals, 66.666667))
            
            
    band = pd.DataFrame()
    band['edge'] = bin_vals

    band['median'] = median_vals
    band['lbnd1'] = lbound1
    band['hbnd1'] = hbound1

    band['lbnd2'] = lbound2
    band['hbnd2'] = hbound2
    
    return band
       

In [ ]:
def update_pair_pc_df(pair_pc_df, modes_of_interest):
    
    df = pair_pc_df.copy()
    
    mode_cols = [f'PC {m}' for m in modes_of_interest]

    df['PC_distance'] = np.sqrt((df[mode_cols]**2).sum(1))

    df['PC_distance_norm'] = (df['PC_distance'] - df['PC_distance'].min()) / (df['PC_distance'].max() - df['PC_distance'].min())
    
    return df

In [ ]:
pair_pc_df = update_pair_pc_df(pair_pc_df, list(range(1)))

In [ ]:
band_df = create_band(pair_pc_df[['PC_distance_norm', 'similarity']], 25)

In [ ]:
ev_line = hv.HLine(np.percentile(pair_pc_df['similarity'], 50))
binned_line = hv.Curve(band_df, ['edge', 'median'])
hex_two_distributions = hv.HexTiles(pair_pc_df[['PC_distance_norm', 'similarity']].to_numpy())
hex_two_distributions.opts(scale=(dim('Count').norm()*0.5)+0.3, width=500, height=500)

overlay = hex_two_distributions * hv.Bivariate(hex_two_distributions) * ev_line * binned_line

overlay.opts(
    opts.HexTiles(min_count=0, width=650, height=500, tools=['hover'], 
                  title=f'{len(pair_pc_df)} pairs, min 365 d. concurrent, Modes {modes_of_interest}',
                  xlabel='Euclidean Distance of PC Scores', ylabel='Similarity [R^2]'),
    opts.Bivariate(show_legend=False, line_width=3),
    opts.HLine(color='red', line_width=3, line_dash='dashed'),
    opts.Curve(color='red', line_width=3, line_dash='solid')
)

## Evaluate Many Combinations of Modes

In [ ]:
def filter_dataframe_by_concurrence(cdf, min_concurrence_len=None, max_concurrence_len=None):
    df = cdf.copy()
    
    if min_concurrence_len is not None:
        df = df[df['concurrent_days'] > 364 * min_concurrence_len]
    if max_concurrence_len is not None:
        df = df[df['concurrent_days'] <= 365 * max_concurrence_len]
    
    return df

In [ ]:
# mode_combos = [[0], [1], [2], [3], [4], [5], [6],
#                [0,1], [0,2], [0,3], [0,4], [0,5], [0,6],
#                [1,2], [1,3], [1,4], [1,5], [1,6],
#                [2,3], [2,4], [2,5], [2,6],
#                [3,4], [3,5], [3,6],
#                [4,5], [4,6],
#                [5,6],
#                [0,1,2], [0,1,3], [0,2,3], [0,2,4], [0,3,4], [0,1,4],
#                [1,2,3], [1,2,4], [1,3,4], [1,3,5], [1,2,5],
#                [0,1,2,3], [0,2,3,4], [1,2,3,4], [0,3,4,5],
#                [0,1,2,3,4], [0,1,2,3,5], [0,1,2,4,5], [0,1,3,4,5], 
#                [0,1,2,3,4,5], [0,1,2,3,5,6], [0,1,2,4,5,6], [0,1,3,4,5,6], 
#               ]

mode_combos = [
    [0], [1], [2], [3], [4], [5], [6], [7], [8], [9],
              [0,1], [1,2], [2,3], [3,4], [4,5], [5,6], [6,7], [7,8], [8,9],
              [0,1,2], [1,2,3], [2,3,4], [3,4,5], [4,5,6], [5,6,7], [6,7,8], [7,8,9],
              [0,1,2,3], [1,2,3,4], [2,3,4,5], [3,4,5,6], [4,5,6,7], [5,6,7,8], [6,7,8,9],
              [0,1,2,3,4], [1,2,3,4,5], [2,3,4,5,6], [3,4,5,6,7], [4,5,6,7,8], [5,6,7,8,9],
              [0,1,2,3,4,5], [1,2,3,4,5,6], [2,3,4,5,6,7], [3,4,5,6,7,8], [4,5,6,7,8,9],
              [0,1,2,3,4,5,6], [1,2,3,4,5,6,7], [2,3,4,5,6,7,8], [3,4,5,6,7,8,9],
              [0,1,2,3,4,5,6,7], [1,2,3,4,5,6,7,8], [2,3,4,5,6,7,8,9],
              [0,1,2,3,4,5,6,7,8], [1,2,3,4,5,6,7,8,9],
              [0,1,2,3,4,5,6,7,8,9,10],
              ]

combo_result_dict, lengths_dict = {}, {}

# set the range of concurrent periods to be evaluated
min_conc, max_conc = 50, None

data = filter_dataframe_by_concurrence(cdf, min_conc, max_conc)
median_similarity = data['similarity'].median()

for c in mode_combos:
    
    PCs, eigvecs, fracVar = do_PCA(data, char_cols)
    
#     print(PCs.shape, len(data))
    
    pc_data = pd.DataFrame(PCs, columns=[f'PC {i}' for i in range(PCs.shape[1])])
#     print(data[data['similarity'].isna()].count())
    pc_df_updated = update_pair_pc_df(pc_data, c)
    
#     print(data['similarity'].isna().sum())
#     print(len(data), len(pc_df))
    
    pc_df_updated['similarity'] = list(data['similarity'].to_numpy())
    
    lengths_dict[str(c)] = {'n_pairs': len(pc_df_updated),
                            'min_max_conc': (min_conc, max_conc)}

    band = create_band(pc_df_updated, 25)
    combo_result_dict[str(c)] = band

In [ ]:
multi_out_df = pd.DataFrame()
for c in combo_result_dict:
    if multi_out_df.empty:
        multi_out_df[f'edge'] = combo_result_dict[c]['edge'].copy()
    multi_out_df[f'median_{c}'] = combo_result_dict[c]['median'].copy()

In [ ]:
# for k, df in combo_result_dict.items():
#     print(df[['edge', 'median']].head())
# combo_result_dict['[0]'].loc[0, 'median']
# unique_mode_lengths = list(set([len(e) for e in combo_result_dict.keys()]))

In [ ]:
# pull together all the lines
line_list = [hv.Path(combo_result_dict[k][['edge', f'median']]) for k, df in combo_result_dict.items()]

# label the lines
label_coords_left = [(-0.0, d.loc[0, 'median']) for k, d in combo_result_dict.items()]
label_coords_right = [(d.loc[len(d)-1, ['edge', 'median']].to_numpy()) for k, d in combo_result_dict.items()]

labels = [e[1:-1] for e in list(combo_result_dict.keys())]

line_labels_left = hv.Labels({('x', 'y'): label_coords_left, 'text': labels}, ['x', 'y'], 'text')
line_labels_right = hv.Labels({('x', 'y'): label_coords_right, 'text': labels}, ['x', 'y'], 'text')

title_info = lengths_dict[list(lengths_dict.keys())[0]]
n_pairs = title_info['n_pairs']
min_dur = title_info['min_max_conc'][0]
max_dur = title_info['min_max_conc'][1]

In [ ]:

ev_line = hv.HLine(median_similarity)

overlay = line_labels_left * hv.Overlay(line_list) * line_labels_right * ev_line

overlay.opts(
    opts.Labels(text_font_size='8pt'),
    opts.Overlay(width=1200, height=900,
              xlabel='Euclidean Distance of PC Scores', 
              ylabel='Similarity [R^2]',
              title=f'{n_pairs} pairs, {min_dur} < Concurrent Period Duration <= {max_dur}',
             ),
    opts.HLine(line_width=4, line_dash='dashed', color='firebrick')
)
# renderer = hv.renderer('bokeh')
# renderer.save(overlay, 'results/img/EV_similarity_vs_PC_diff_distance_curves')


## How to Describe the Coefficient of Determination

The quality of a numerical model is described by how well its predictions correlate with observation.  In hydrology, the coefficient of determination (COD) is often used to describe how well synchronized is the timing and volume of flow in two different rivers.  The COD indicating perfect correlation is one, and zero represents total randomness.  In between is a continuum of values that don't mean much on their own but begin to express something when gathered in large numbers.

But what exactly does a COD of 0.74 mean?  Why would an author include the extra 0.04 precision when the decision associated with its evaluation is probably closer to binary?  *Yes, this correlation is good enough to use to base my design input on; No, some other method or source of information is required in order to produce a useful value*.  What is the lowest value a scientist or engineer is willing to tolerate, to publish, or to use to form the basis of some critical design component?  The answer is of course relative to the question being asked of the data.  Relationships between sets of observations are described as *strong* or *weak*, they are *highly*, *fairly well* or *not well* correlated.  If you want to get creative, [there is a website](https://describingwords.io/for/correlation) with *329+ adjectives for correlation, ordered by usage frequency*.  @Moed_2017 offers the sound advice to "*never take a correlation coefficient for granted and never consider merely the numerical outcome.*"

A particularly routine exercise for practitioners in the resource industry is the daily average flow correlation.  Characterization of runoff is often required in ungauged basins.  Long-term records are sought from nearby gauging stations to find the most representative proxy for runoff at the location of interest.  Qualitative comparisons are written to justify choosing one candidate proxy over another.  The comparisons are based on proximity, basin characteristics such as elevation, topography, geology, land use, etc.  Quantitative justification for a proxy is uncommon, because quantitative measures of basin characteristics are often unailable or are otherwise difficult to determine.  

It has been shown **(citation)** that some basin characteristics are more important than others as far as indicating the strength of the relationship of runoff between two different basins.  

The figures above describe a relationship between basin characteristics and the COD.  Basin characteristics are compared by a distance measure defined by the euclidean distance of the principal component scores over the number of modes describing **X%** of the variance in the data.  The figure above-left suggests that for a basin pairs to have a COD greater than 0.8, the distance measure is beyond 2 standard deviations from the expected value of the entire sample, where the sample is made up of COD values of all possible pairs of station records that have a minimum 40 years of concurrent record.  The expected value is then a baseline of a random selection from all possible proxy basins.

The figure above-right describes the situation where the basin characteristics are known but one or both rivers are ungauged.  The figure suggests that only the smallest of distance measure falls outside of 2 standard deviations from the expected value of all comparisons.  Based on basin characteristics alone, if the distance measure (standard or principal component score) is greater than roughly 0.2, then a random selection of two basins is expected to produce a better correlation.  The distance measure of approximately 0.2 defines a baseline bound of random pair selection.  Note that a random pairing (doesn't appear to be) the same as a random selection of a proxy.  

Can a tool be devised to take as input a set of basin characteristics and output the distribution of COD given all available data?  

-how does the expected value change as a function of record length (i.e. here 40 years is used, but what if analysis is undertaken by concurrence length ranges?  i.e. 1 year, 2, 3, 5, 10, 20, 40 years?  

## Run ICA

Find the modes of maximum statistical independence.

In [ ]:
# do ICA using built-in library
x = df.index.values
y = df.to_numpy()

n_modes = np.shape(y)[1] #dimension of input = 4, so want 4 modes

ica = FastICA(n_components=n_modes)
S_ = ica.fit_transform(y)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
assert np.allclose(y, np.dot(S_, A_.T) + ica.mean_)

# ica = FastICA()
# S_ica_ = ica.fit(y).transform(y)
S_ /= S_.std(axis=0)

In [ ]:
ica_df = pd.DataFrame(S_, columns=[f'IC {e}' for e in range(PCs.shape[1])])
ica_component_grid = create_grid_plot(ica_df)

show(gridplot(ica_component_grid, ncols=4, plot_width=200, plot_height=130))

In [ ]:
ica_space_figs = make_PC_gridplot(S_, 9)
show(gridplot(ica_space_figs, ncols=6, plot_width=200, plot_height=130))

In [ ]:
#run SOM -- this code creates/trains the SOM and calculates stats of interest

nx = 3
ny = 3

d = y



#make, initialize, and train the SOM
print(d.shape)
som = MiniSom(nx, ny, data.shape[1], sigma=1, learning_rate=0.5) # initialization of (ny x nx) SOM
som.pca_weights_init(data)
som.train_random(data, 100) # trains the SOM with 100 iterations

qnt = som.quantization(data) #this is the pattern of the BMU of each observation (ie: has same size as data input to SOM)
bmu_patterns = som.get_weights() #this is the pattern of each BMU; size = (nx, ny, len(data[0]))
QE = som.quantization_error(data) #quantization error of map
TE = som.topographic_error(data) #topographic error of map

#calculate the BMU of each observation
bmus = []
for kk in range(len(data)):
    bmus.append(som.winner(data[kk]))
    
#inds gives the sequential coordinates of each SOM node (useful for plotting)
inds = []
for ii in range(ny):
    for jj in range(nx):
        inds.append((ii,jj))

# compute the frequency of each BMU
freq = np.zeros((nx,ny))
for bmu in bmus:
    freq[bmu[0]][bmu[1]]+=1
freq/=len(data)

In [ ]:
#visualize
from matplotlib import pylab

fig, ax = plt.subplots(ny, nx, figsize=(2*nx,3*ny))
row, col = 0, 0
for kk in range(nx*ny):   
    indx = inds[kk][1]
    indy = inds[kk][0]
#     plt.imshow(np.reshape(bmu_patterns[indx][indy],(faceH,faceW)).T,cmap='gray')
    ax[col, row].plot(np.reshape(bmu_patterns[indx][indy], (17)).T)
    ax[col, row].set_title('Freq = ' + str(freq[indx][indy]*100)[:4] + '%')

#     ax[col, row].set_ylabel('Precipitation (Normalized)')
#     ax[col, row].set_xlabel('Percentage of Basin Area')
    col += 1
    if col == ny:
        col = 0
        row += 1
    
    
plt.tight_layout()

In [ ]:
#now, loop through and create a range of sizes of SOMs and compare TE and QE 

ny_array = [2,2,3,4,5,4,5,5,6,7,8, 9, 10, 11]
nx_array = [2,3,3,3,3,4,4,5,6,7,8, 9, 10, 11]

QE = []
TE = []
for kk in range(len(ny_array)):
    nx = nx_array[kk]
    ny = ny_array[kk]
    
    #make, initialize, and train the SOM
    data = df.to_numpy()
    som = MiniSom(nx, ny, 17, sigma=0.3, learning_rate=0.4) # initialization of SOM
    som.pca_weights_init(data)
    som.train_random(data, 100) # trains the SOM with 100 iterations

    qnt = som.quantization(data) #this is the pattern of the BMU of each observation (ie: has same size as data input to SOM)
    bmu_patterns = som.get_weights() #this is the pattern of each BMU; size = (nx, ny, len(data[0]))
    QE.append(som.quantization_error(data)) #quantization error of map
    TE.append(som.topographic_error(data)) #topographic error of map

In [ ]:
#visualize QE and TE (two different y axes)

fig, ax1= plt.subplots()

#QE
color = 'tab:red'
ax1.set_ylabel('QE',color=color)
ax1.plot(QE,color=color)
ax1.tick_params(axis='y',labelcolor=color)

#TE
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('TE',color=color)
ax2.plot(TE,color=color)
ax2.tick_params(axis='y',labelcolor=color)

plt.title('QE and TE for Different Map Sizes')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
nx, ny = 8, 8

som = somoclu.Somoclu(nx, ny, compactsupport=False)
%time som.train(data)

In [ ]:
som.view_component_planes()

In [ ]:
colors = ["red"] * len(data)
colors.extend(["green"] * len(data))
colors.extend(["blue"] * len(data))

In [ ]:
som.view_umatrix(bestmatches=True, bestmatchcolors=colors, labels=list(range(len(data))))

In [ ]:
som = somoclu.Somoclu(nx, ny, maptype="toroid",
                      compactsupport=False)
som.train(data)
som.view_umatrix(bestmatches=True, bestmatchcolors=colors)

In [ ]:
som = somoclu.Somoclu(nx, ny, gridtype="hexagonal",
                      compactsupport=False)
som.train(data)
som.view_umatrix(bestmatches=True, bestmatchcolors=colors)

In [ ]:
som = somoclu.Somoclu(nx, ny, maptype="toroid",
                      compactsupport=False, initialization="pca")
som.train(data)
som.view_umatrix(bestmatches=True, bestmatchcolors=colors)

In [ ]:
som.cluster()
som.view_umatrix(bestmatches=True)